## Day 85 Lecture 1 Assignment

In this assignment, we will learn how to use the other layers to improve our model performance.

In [1]:
import numpy as np
import pandas as pd

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

*We* will explore a dataset containing information about twitter users and will detect whether or not the user is a bot.

In [3]:
twitter = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/training_data_2_csv_UTF.csv')

In [4]:
twitter.head()

,id,id_str,screen_name,location,description,url,followers_count,friends_count,listed_count,created_at,favourites_count,verified,statuses_count,lang,status,default_profile,default_profile_image,has_extended_profile,name,bot
0,8.160000e+17,"""815745789754417152""","""HoustonPokeMap""","""Houston, TX""","""Rare and strong PokŽmon in Houston, TX. See m...","""https://t.co/dnWuDbFRkt""",1291,0,10,"""Mon Jan 02 02:25:26 +0000 2017""",0,False,78554,"""en""","{\r ""created_at"": ""Sun Mar 12 15:44:04 +0...",True,False,False,"""Houston PokŽ Alert""",1
1,4.843621e+09,4843621225,kernyeahx,"Templeville town, MD, USA",From late 2014 Socium Marketplace will make sh...,NaN,1,349,0,2/1/2016 7:37,38,False,31,en,NaN,True,False,False,Keri Nelson,1
2,4.303727e+09,4303727112,mattlieberisbot,NaN,"Inspired by the smart, funny folks at @replyal...",https://t.co/P1e1o0m4KC,1086,0,14,Fri Nov 20 18:53:22 +0000 2015,0,False,713,en,"{'retweeted': False, 'is_quote_status': False,...",True,False,False,Matt Lieber Is Bot,1
3,3.063139e+09,3063139353,sc_papers,NaN,NaN,NaN,33,0,8,2/25/2015 20:11,0,False,676,en,Construction of human anti-tetanus single-chai...,True,True,False,single cell papers,1
4,2.955142e+09,2955142070,lucarivera16,"Dublin, United States",Inspiring cooks everywhere since 1956.,NaN,11,745,0,1/1/2015 17:44,146,False,185,en,NaN,False,False,False,lucarivera16,1


In [5]:
twitter.shape

(2797, 20)

Start by getting rid of all columns that are not useful.

In [6]:
# Answer below:

twitter.drop(columns=['id', 'id_str','screen_name','created_at', 'url', 'status', 'name',], inplace=True)


Next, get rid of all columns that contain more than 30% missing data. After that, remove all rows containing at least one missing observation.

In [7]:
# Answer below:
twitter.isnull().sum()/twitter.isnull().count()


location                 0.364676
description              0.144083
followers_count          0.000000
friends_count            0.000000
listed_count             0.000000
favourites_count         0.000000
verified                 0.000000
statuses_count           0.000000
lang                     0.000000
default_profile          0.000000
default_profile_image    0.000000
has_extended_profile     0.035395
bot                      0.000000
dtype: float64

In [8]:
twitter.drop(columns=['location'], inplace=True)

twitter.isnull().sum()/twitter.isnull().count()

description              0.144083
followers_count          0.000000
friends_count            0.000000
listed_count             0.000000
favourites_count         0.000000
verified                 0.000000
statuses_count           0.000000
lang                     0.000000
default_profile          0.000000
default_profile_image    0.000000
has_extended_profile     0.035395
bot                      0.000000
dtype: float64

In [9]:
twitter.dropna(inplace=True)

twitter.isnull().sum()/twitter.isnull().count()

description              0.0
followers_count          0.0
friends_count            0.0
listed_count             0.0
favourites_count         0.0
verified                 0.0
statuses_count           0.0
lang                     0.0
default_profile          0.0
default_profile_image    0.0
has_extended_profile     0.0
bot                      0.0
dtype: float64

In [10]:
twitter.head()

,description,followers_count,friends_count,listed_count,favourites_count,verified,statuses_count,lang,default_profile,default_profile_image,has_extended_profile,bot
0,"""Rare and strong PokŽmon in Houston, TX. See m...",1291,0,10,0,False,78554,"""en""",True,False,False,1
1,From late 2014 Socium Marketplace will make sh...,1,349,0,38,False,31,en,True,False,False,1
2,"Inspired by the smart, funny folks at @replyal...",1086,0,14,0,False,713,en,True,False,False,1
4,Inspiring cooks everywhere since 1956.,11,745,0,146,False,185,en,False,False,False,1
5,Just a guy trying to do good by telling everyo...,1,186,0,0,False,11,en,True,False,True,1


Now we will use our embedding functions from a previous assignment.

In [11]:
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def remove_stopwords(input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words)       

def stem_list(word_list):
    stemmed = []
    for word in word_list:
        stemmedword = stemmer.stem(word)
        stemmed.append(stemmedword)
    return stemmed

def normalize(terms):
    terms = terms.lower()
    terms = remove_stopwords(terms)
    word_delimiters = u'[\\[\\]\n.!?,;:\t\\-\\"\\(\\)\\\'\u2019\u2013 ]'
    term_list = re.split(word_delimiters, terms)
    trimmed = [x.rstrip() for x in term_list]
    stemmed = stem_list(trimmed)
    space = ' '
    normed = space.join(stemmed)
    normed = normed.replace('  ', ' ')
    return normed

We will create two branches, one branch will process the text data in the description and the other will process all other columns. First, create a numpy array with the encoded data from the description column. Normalize each description, one hot encode the text, pad the row and create a numpy array.

In [12]:
# Answer below:

twitter['description_norm'] = twitter.description.apply(normalize)

In [13]:
vocab_size = len(set(twitter.description_norm.sum()))
vocab_size

118

In [14]:
from tensorflow.keras.preprocessing.text import one_hot

description_encoded = twitter.description_norm.apply(one_hot, args=[vocab_size])
description_encoded

0               [88, 87, 4, 67, 80, 44, 4, 7, 24, 67, 61]
1       [8, 83, 95, 59, 9, 114, 11, 55, 105, 32, 37, 8...
2       [58, 72, 113, 45, 116, 113, 7, 53, 57, 93, 99,...
4                                    [58, 64, 98, 41, 42]
5                     [106, 91, 101, 51, 117, 31, 52, 72]
                              ...                        
2792               [102, 9, 99, 2, 51, 82, 6, 14, 78, 84]
2793    [26, 91, 41, 5, 23, 54, 31, 12, 41, 5, 21, 46,...
2794                                                 [40]
2795    [41, 5, 26, 9, 78, 96, 93, 24, 17, 76, 24, 2, ...
2796                     [78, 96, 58, 29, 91, 78, 86, 86]
Name: description_norm, Length: 2355, dtype: object

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

description_vars = pad_sequences(description_encoded)
description_vars.shape

(2355, 66)

Convert all boolean variables to numeric (zero for false and 1 for true)

In [16]:
twitter.dtypes

description              object
followers_count           int64
friends_count             int64
listed_count              int64
favourites_count          int64
verified                   bool
statuses_count            int64
lang                     object
default_profile            bool
default_profile_image      bool
has_extended_profile     object
bot                       int64
description_norm         object
dtype: object

In [17]:
# Answer below:
twitter.has_extended_profile = pd.to_numeric(twitter.has_extended_profile) * 1


In [18]:
twitter.verified = twitter.verified * 1

In [19]:
twitter.default_profile = twitter.default_profile * 1
twitter.default_profile_image = twitter.default_profile_image * 1

In [20]:
twitter.dtypes

description              object
followers_count           int64
friends_count             int64
listed_count              int64
favourites_count          int64
verified                  int64
statuses_count            int64
lang                     object
default_profile           int64
default_profile_image     int64
has_extended_profile      int64
bot                       int64
description_norm         object
dtype: object

In [21]:
twitter.head()

,description,followers_count,friends_count,listed_count,favourites_count,verified,statuses_count,lang,default_profile,default_profile_image,has_extended_profile,bot,description_norm
0,"""Rare and strong PokŽmon in Houston, TX. See m...",1291,0,10,0,0,78554,"""en""",1,0,0,1,rare strong pokžmon houston tx see pokžmon ht...
1,From late 2014 Socium Marketplace will make sh...,1,349,0,38,0,31,en,1,0,0,1,late 2014 socium marketplac make shop fundamen...
2,"Inspired by the smart, funny folks at @replyal...",1086,0,14,0,0,713,en,1,0,0,1,inspir smart funni folk @replyal @gimletmedia ...
4,Inspiring cooks everywhere since 1956.,11,745,0,146,0,185,en,0,0,0,1,inspir cook everywher sinc 1956
5,Just a guy trying to do good by telling everyo...,1,186,0,0,0,11,en,1,0,1,1,guy tri good tell everyon els get better


Create dummy variables out of the relevant object columns. Take caution when converting columns that may incorrectly classified as object.

In [22]:
# Answer below:

twitter_one_hot = pd.get_dummies(twitter, columns=['lang'], drop_first=True)
twitter_one_hot.head()

,description,followers_count,friends_count,listed_count,favourites_count,verified,statuses_count,default_profile,default_profile_image,has_extended_profile,bot,description_norm,"lang_""en""","lang_""en-gb""","lang_""es""","lang_""fr""","lang_""it""","lang_""ja""","lang_""ko""","lang_""pt""","lang_""zh-CN""","lang_""zh-cn""",lang_ar,lang_de,lang_en,lang_en-gb,lang_es,lang_fr,lang_gl,lang_it,lang_ja,lang_nl,lang_pt,lang_ru,lang_ta,lang_th,lang_tr,lang_zh-CN,lang_zh-cn
0,"""Rare and strong PokŽmon in Houston, TX. See m...",1291,0,10,0,0,78554,1,0,0,1,rare strong pokžmon houston tx see pokžmon ht...,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,From late 2014 Socium Marketplace will make sh...,1,349,0,38,0,31,1,0,0,1,late 2014 socium marketplac make shop fundamen...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Inspired by the smart, funny folks at @replyal...",1086,0,14,0,0,713,1,0,0,1,inspir smart funni folk @replyal @gimletmedia ...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Inspiring cooks everywhere since 1956.,11,745,0,146,0,185,0,0,0,1,inspir cook everywher sinc 1956,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Just a guy trying to do good by telling everyo...,1,186,0,0,0,11,1,0,1,1,guy tri good tell everyon els get better,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Min max scale the data decribing each user (do not min max scale the word embeddings).

In [23]:
# Answer below:
from sklearn.preprocessing import MinMaxScaler

scale = MinMaxScaler()

counts = twitter_one_hot.filter(like='_count')
counts_scaled = pd.DataFrame(scale.fit_transform(counts), columns=counts.columns)



user_vars = pd.concat([counts_scaled, twitter_one_hot.drop(columns=counts.columns).reset_index()], axis=1)
user_vars.drop(columns=['description', 'description_norm', 'index'], inplace=True)
user_vars

,followers_count,friends_count,listed_count,favourites_count,statuses_count,verified,default_profile,default_profile_image,has_extended_profile,bot,"lang_""en""","lang_""en-gb""","lang_""es""","lang_""fr""","lang_""it""","lang_""ja""","lang_""ko""","lang_""pt""","lang_""zh-CN""","lang_""zh-cn""",lang_ar,lang_de,lang_en,lang_en-gb,lang_es,lang_fr,lang_gl,lang_it,lang_ja,lang_nl,lang_pt,lang_ru,lang_ta,lang_th,lang_tr,lang_zh-CN,lang_zh-cn
0,1.340302e-05,0.000000,0.000016,0.000000,0.011445,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.038189e-08,0.000354,0.000000,0.000053,0.000005,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.127473e-05,0.000000,0.000023,0.000000,0.000104,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.142008e-07,0.000756,0.000000,0.000204,0.000027,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.038189e-08,0.000189,0.000000,0.000000,0.000002,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2350,1.972352e-04,0.002034,0.000685,0.003505,0.000510,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2351,3.322205e-07,0.000055,0.000000,0.000001,0.000014,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2352,4.676464e-01,0.007557,0.109872,0.000034,0.001400,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2353,1.661103e-07,0.000065,0.000002,0.000021,0.000009,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [24]:
target = user_vars.bot
user_vars.drop(columns=['bot'], inplace=True)

In [25]:
user_vars.dtypes

followers_count          float64
friends_count            float64
listed_count             float64
favourites_count         float64
statuses_count           float64
verified                   int64
default_profile            int64
default_profile_image      int64
has_extended_profile       int64
lang_"en"                  uint8
lang_"en-gb"               uint8
lang_"es"                  uint8
lang_"fr"                  uint8
lang_"it"                  uint8
lang_"ja"                  uint8
lang_"ko"                  uint8
lang_"pt"                  uint8
lang_"zh-CN"               uint8
lang_"zh-cn"               uint8
lang_ar                    uint8
lang_de                    uint8
lang_en                    uint8
lang_en-gb                 uint8
lang_es                    uint8
lang_fr                    uint8
lang_gl                    uint8
lang_it                    uint8
lang_ja                    uint8
lang_nl                    uint8
lang_pt                    uint8
lang_ru   

Now we'll create the two branches. Create a model for the numeric data that consists of 3 dense layers. An input layer and two hidden layers of size 32.

In [26]:
user_vars.shape

(2355, 36)

In [27]:
# Answer below:

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Embedding, LSTM, concatenate

input_layer1 = Input(shape=user_vars.shape[1],)
h1 = Dense(32, activation='relu')(input_layer1)
h2 = Dense(32, activation='relu')(h1)

Create the second branch of the model using the encoded words. This branch will consist of 4 layers: An input layer, an embedding layer returning data of dimension 100, an LSTM layer of unit size 32 and a dense layer of unit size 32. 

In [28]:
vocab_size

118

In [29]:
np.max(description_vars) + 1

118

In [30]:
description_vars.shape

(2355, 66)

In [31]:
# Answer below:
input_layer2 = Input(shape=(description_vars.shape[1], ))
embedding = Embedding(np.max(description_vars) + 1, 100)(input_layer2)
lstm = LSTM(32)(embedding)
dense = Dense(32, activation='relu')(lstm)


Merge the two models using the `concatenate` function (merge the two final dense layers in each branch) and create an output dense layer.

In [32]:
# Answer below:
concat = concatenate([h2, dense])
output = Dense(1, activation='sigmoid')(concat)


Create a model using the two inputs and the single output and print the summary

In [33]:
# Answer below: 

model = Model(inputs=[input_layer1, input_layer2], outputs=output)

model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 66)]         0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 36)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 66, 100)      11800       input_2[0][0]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 32)           1184        input_1[0][0]                    
______________________________________________________________________________________________

Compile and fit the model using the appropriate optimizer, loss, and metrics. Train the model for 10 epochs with a batch size of 128.

In [36]:
# Answer below:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x=[user_vars, description_vars], y=target, epochs=10, batch_size=128)

Epoch 1/10
19/19 [==============================] - 2s 9ms/step - loss: 0.3306 - accuracy: 0.8562
Epoch 2/10
19/19 [==============================] - 0s 9ms/step - loss: 0.3074 - accuracy: 0.8616
Epoch 3/10
19/19 [==============================] - 0s 9ms/step - loss: 0.3072 - accuracy: 0.8658
Epoch 4/10
19/19 [==============================] - 0s 9ms/step - loss: 0.2965 - accuracy: 0.8682
Epoch 5/10
19/19 [==============================] - 0s 9ms/step - loss: 0.3181 - accuracy: 0.8549
Epoch 6/10
19/19 [==============================] - 0s 9ms/step - loss: 0.3033 - accuracy: 0.8709
Epoch 7/10
19/19 [==============================] - 0s 8ms/step - loss: 0.2989 - accuracy: 0.8723
Epoch 8/10
19/19 [==============================] - 0s 8ms/step - loss: 0.2979 - accuracy: 0.8699
Epoch 9/10
19/19 [==============================] - 0s 8ms/step - loss: 0.2765 - accuracy: 0.8823
Epoch 10/10
19/19 [==============================] - 0s 9ms/step - loss: 0.2884 - accuracy: 0.8706
